# 한정된 데이터로 Transfer learning 적용해보기

# 1. CT이미지 데이터셋 살펴보기

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import torch

In [1]:
import torch

In [4]:
data_dir = "./DATASET/Segmentation/"
data_df = pd.read_csv(os.path.join(data_dir, "train.csv"))

In [5]:
data_df

,ImageId,MaskId
0,ID00007637202177411956430_0.jpg,ID00007637202177411956430_mask_0.jpg
1,ID00007637202177411956430_1.jpg,ID00007637202177411956430_mask_1.jpg
2,ID00007637202177411956430_2.jpg,ID00007637202177411956430_mask_2.jpg
3,ID00007637202177411956430_3.jpg,ID00007637202177411956430_mask_3.jpg
4,ID00007637202177411956430_4.jpg,ID00007637202177411956430_mask_4.jpg
...,...,...
16703,ID00426637202313170790466_403.jpg,ID00426637202313170790466_mask_403.jpg
16704,ID00426637202313170790466_404.jpg,ID00426637202313170790466_mask_404.jpg
16705,ID00426637202313170790466_405.jpg,ID00426637202313170790466_mask_405.jpg
16706,ID00426637202313170790466_406.jpg,ID00426637202313170790466_mask_406.jpg


In [6]:
def extract_client_id(x):
    return x.split('-')[0]

In [7]:
data_df['id'] = data_df.ImageId.apply(lambda x: extract_client_id(x))
data_df

,ImageId,MaskId,id
0,ID00007637202177411956430_0.jpg,ID00007637202177411956430_mask_0.jpg,ID00007637202177411956430_0.jpg
1,ID00007637202177411956430_1.jpg,ID00007637202177411956430_mask_1.jpg,ID00007637202177411956430_1.jpg
2,ID00007637202177411956430_2.jpg,ID00007637202177411956430_mask_2.jpg,ID00007637202177411956430_2.jpg
3,ID00007637202177411956430_3.jpg,ID00007637202177411956430_mask_3.jpg,ID00007637202177411956430_3.jpg
4,ID00007637202177411956430_4.jpg,ID00007637202177411956430_mask_4.jpg,ID00007637202177411956430_4.jpg
...,...,...,...
16703,ID00426637202313170790466_403.jpg,ID00426637202313170790466_mask_403.jpg,ID00426637202313170790466_403.jpg
16704,ID00426637202313170790466_404.jpg,ID00426637202313170790466_mask_404.jpg,ID00426637202313170790466_404.jpg
16705,ID00426637202313170790466_405.jpg,ID00426637202313170790466_mask_405.jpg,ID00426637202313170790466_405.jpg
16706,ID00426637202313170790466_406.jpg,ID00426637202313170790466_mask_406.jpg,ID00426637202313170790466_406.jpg


In [8]:
def get_client_data(data_df, index):
    client_ids = np.unique(data_df.id.values)
    client_id = client_ids[index]
    client_data = data_df[data_df.id == client_id]
    Image_files = list(client_data['ImageId'])
    mask_files = list(client_data['MaskId'])
    return client_id, Image_files, mask_files

In [9]:
index = 100
get_client_data(data_df, index)

('ID00009637202177434476278_161.jpg',
 ['ID00009637202177434476278_161.jpg'],
 ['ID00009637202177434476278_mask_161.jpg'])

In [10]:
regions = ["background", "trachea", "heart", "lung"]
colors = ((0,0,0), (255,0,0), (0,255,0), (0,0,255))

In [11]:
index = 50
client_id, image_files , mask_files = get_client_data(data_df, index)

canvas = np.zeros(shape=(512, 2*512+50, 3), dtype=np.uint8)
for i in range(len(image_files)):
    image = cv2.imread(os.path.join(data_dir, "images", image_files[i]))
    mask = cv2.imread(os.path.join(data_dir, "masks", mask_files[i]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    mask[mask<240] = 0
    mask[mask>=240] = 255
    
    canvas[:, 0:512, :] = image
    canvas[:, 512+50:2*512+50, :] = mask
    
    cv2.imshow('image', canvas)
    key = cv2.waitKey(100)
    if key == 27:
        break
    if key == ord('s'):
        cv2.waitKey(0)
cv2.destroyAllWindows()

# 2. 데이터셋 구축과 연산을 위한 텐서변환 모듈 작성하기

In [12]:
IMAGE_SIZE = 224

In [19]:
class CT_dataset():
    def __init__(self, data_dir, phase, transformer=None):
        self.phase = phase
        self.images_dir = os.path.join(data_dir, phase, 'images')
        self.masks_dir = os.path.join(data_dir, phase, 'masks')
        self.image_files = [filename for filename in os.listdir(self.images_dir) if filename.endswith('jpg')]
        self.mask_files = [filename for filename in os.listdir(self.masks_dir) if filename.endswith('jpg')]
        assert len(self.image_files) == len(self.mask_files)
        
        self.transformer = transformer
        
    def __len__(self,):
        return len(self.image_files)
    
    def __getitem__(self, index):
        image = cv2.imread(os.path.join(self.images_dir, self.image_files[index]))
        image = cv2.resize(image, dsize=(IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_LINEAR)
        mask = cv2.imread(os.path.join(self.masks_dir, self.mask_files[index]))
        mask = cv2.resize(mask, dsize=(IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
        
        mask[mask < 240] = 0
        mask[mask >= 240] = 255
        mask = mask / 255
        
        mask_H, mask_W, mask_C = mask.shape
        background = np.ones(shape=(mask_H, mask_W))
        background[mask[..., 0] != 0] = 0
        background[mask[..., 1] != 0] = 0
        background[mask[..., 2] != 0] = 0
        
        mask = np.concatenate([np.expand_dims(background, axis=-1), mask], axis=-1)
        """
        background shape: (H, W) -> shape 확장 (H, W, 1)로
        mask shape: (H, W, C)
        
        new mask shape: (H, W, C+1)
        """
        mask = np.argmax(mask, axis=-1)
        "new mask shape: (H, W, 4) -> new mask shape: (H, W)"
        
        if self.transformer:
            image = self.transformer(image)
            
        target = torch.from_numpy(mask).long()
        
        return image, target

In [20]:
dset = CT_dataset(data_dir, "train")

In [22]:
image, target = dset[0]
image.shape, target.shape

((224, 224, 3), torch.Size([224, 224]))

In [23]:
from torchvision import transforms

def build_transformer():
    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229, 0.224, 0.225])
    ])
    return transformer

In [24]:
def collate_fn(batch):
    images = []
    targets = []
    for a, b in batch:
        images.append(a)
        targets.append(b)
    images = torch.stack(images, dim=0)
    targets = torch.stack(targets, dim=0)
    return images, targets

In [25]:
transformer = build_transformer()
dset = CT_dataset(data_dir=data_dir, phase="train", transformer=transformer)

In [26]:
image, target = dset[0]
print(f"image shape: {image.shape}")
print(f"target shape: {target.shape}")

image shape: torch.Size([3, 224, 224])
target shape: torch.Size([224, 224])


In [27]:
from torch.utils.data import DataLoader

In [28]:
dloader = DataLoader(dset, batch_size=4, shuffle=True, collate_fn=collate_fn)

In [29]:
for index, batch in enumerate(dloader):
    images = batch[0]
    targets = batch[1]
    print(f"images shape: {images.shape}")
    print(f"targets shape: {targets.shape}")
    
    if index == 0:
        break

images shape: torch.Size([4, 3, 224, 224])
targets shape: torch.Size([4, 224, 224])


In [30]:
def build_dataloader(data_dir, batch_size=4):
    transformer = build_transformer()
    
    dataloaders = {}
    train_dataset = CT_dataset(data_dir=data_dir, phase="train", transformer=transformer)
    dataloaders["train"] = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    
    val_dataset = CT_dataset(data_dir=data_dir, phase="val", transformer=transformer)
    dataloaders["val"] = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    return dataloaders

In [31]:
dataloaders = build_dataloader(data_dir=data_dir)

for phase in ["train", "val"]:
    for index, batch in enumerate(dataloaders[phase]):
        images = batch[0]
        targets = batch[1]
        print(f"images shape: {images.shape}")
        print(f"targets shape: {targets.shape}")
        
        if index == 0:
            break

images shape: torch.Size([4, 3, 224, 224])
targets shape: torch.Size([4, 224, 224])
images shape: torch.Size([4, 3, 224, 224])
targets shape: torch.Size([4, 224, 224])


# 3. VGG16 Backbone을 이용한 U-Net 아키텍쳐 구현해보기

In [32]:
import torch.nn as nn

In [33]:
def ConvLayer(in_channels, out_channels, kernel_size=3, padding=1):
    layers = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    
        nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    )
    return layers

In [42]:
def UpConvLayer(in_channels, out_channels):
    layers = nn.Sequential(
        nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )
    return layers

In [35]:
from torchvision import models

In [36]:
vgg16 = models.vgg16_bn(pretrained=False)
vgg16

/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [37]:
class Encoder(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        
        backbone = models.vgg16_bn(pretrained=pretrained).features
        self.conv_block1 = nn.Sequential(*backbone[:6])
        self.conv_block2 = nn.Sequential(*backbone[6:13])
        self.conv_block3 = nn.Sequential(*backbone[13:20])
        self.conv_block4 = nn.Sequential(*backbone[20:27])
        self.conv_block5 = nn.Sequential(*backbone[27:34],
                                        ConvLayer(512, 1024, kernel_size=1, padding=0))
        
    def forward(self, x):
        encode_features = []
        out = self.conv_block1(x)
        encode_features.append(out)
        
        out = self.conv_block2(out)
        encode_features.append(out)
        
        out = self.conv_block3(out)
        encode_features.append(out)
        
        out = self.conv_block4(out)
        encode_features.append(out)
        
        out = self.conv_block5(out)
        return out, encode_features

In [38]:
encoder = Encoder(pretrained=False)
x = torch.randn(1, 3, 224, 224)
out, ftrs = encoder(x)

/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


In [39]:
for ftr in ftrs:
    print(ftr.shape)
print(out.shape)

torch.Size([1, 64, 224, 224])
torch.Size([1, 128, 112, 112])
torch.Size([1, 256, 56, 56])
torch.Size([1, 512, 28, 28])
torch.Size([1, 1024, 14, 14])


In [45]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.upconv_layer1 = UpConvLayer(in_channels=1024, out_channels=512)
        self.conv_block1 = ConvLayer(in_channels=512+512, out_channels=512)
        
        self.upconv_layer2 = UpConvLayer(in_channels=512, out_channels=256)
        self.conv_block2 = ConvLayer(in_channels=256+256, out_channels=256)
        
        self.upconv_layer3 = UpConvLayer(in_channels=256, out_channels=128)
        self.conv_block3 = ConvLayer(in_channels=128+128, out_channels=128)
        
        self.upconv_layer4 = UpConvLayer(in_channels=128, out_channels=64)
        self.conv_block4 = ConvLayer(in_channels=64+64, out_channels=64)
        
    def forward(self, x, encoder_features):
        out = self.upconv_layer1(x)
        out = torch.cat([out, encoder_features[-1]], dim=1)
        out = self.conv_block1(out)
        
        out = self.upconv_layer2(out)
        out = torch.cat([out, encoder_features[-2]], dim=1)
        out = self.conv_block2(out)
        
        out = self.upconv_layer3(out)
        out = torch.cat([out, encoder_features[-3]], dim=1)
        out = self.conv_block3(out)
        
        out = self.upconv_layer4(out)
        out = torch.cat([out, encoder_features[-4]], dim=1)
        out = self.conv_block4(out)

        return out

In [46]:
encoder = Encoder(pretrained=False)
decoder = Decoder()
x = torch.randn(1, 3, 224, 224)
out, ftrs = encoder(x)
out = decoder(out, ftrs)

In [47]:
print(out.shape)

torch.Size([1, 64, 224, 224])


In [48]:
class UNet(nn.Module):
    def __init__(self, num_classes, pretrained):
        super().__init__()
        self.encoder = Encoder(pretrained)
        self.decoder = Decoder()
        self.head = nn.Conv2d(64, num_classes, kernel_size=1)
        
    def forward(self, x):
        out, encode_features = self.encoder(x)
        out = self.decoder(out, encode_features)
        out = self.head(out)
        return out

In [49]:
model = UNet(num_classes=4, pretrained=False)
x = torch.randn(1, 3, 224, 224)
out = model(x)

In [50]:
print(out.shape)

torch.Size([1, 4, 224, 224])


# 4. Semantic segmentation Loss와 학습코드 작성하기

In [51]:
import torch.nn.functional as F

In [52]:
class UNet_metric():
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.CE_loss = nn.CrossEntropyLoss(reduction="mean")
        
    def __call__(self, pred, target):
        loss1 = self.CE_loss(pred, target)
        onehot_pred = F.one_hot(torch.argmax(pred, dim=1), num_classes=self.num_classes).permute(0, 3, 1, 2)
        onehot_target = F.one_hot(target, num_classes=self.num_classes).permute(0, 3, 1, 2)
        loss2 = self._get_dice_loss(onehot_pred, onehot_target)
        loss = loss1 + loss2
        
        dice_coefficient = self._get_batch_dice_coefficient(onehot_pred, onehot_target)
        return loss, dice_coefficient
    
    def _get_dice_coeffient(self, pred, target):
        set_inter = torch.dot(pred.reshape(-1).float(), target.reshape(-1).float())
        set_sum = pred.sum() + target.sum()
        if set_sum.item() == 0:
            set_sum = 2 * set_inter
        dice_coeff = (2 * set_inter) / (set_sum + 1e-9)
        return dice_coeff
    
    def _get_multiclass_dice_coefficient(self, pred, target):
        dice = 0
        for class_index in range(1, self.num_classes):
            dice += self._get_dice_coeffient(pred[class_index], target[class_index])
        return dice / (self.num_classes - 1)
    
    def _get_batch_dice_coefficient(self, pred, target):
        num_batch = pred.shape[0]
        dice = 0
        for batch_index in range(num_batch):
            dice += self._get_multiclass_dice_coefficient(pred[batch_index], target[batch_index])
        return dice / num_batch
    
    def _get_dice_loss(self, pred, target):
        return 1 - self._get_batch_dice_coefficient(pred, target)

In [53]:
def train_one_epoch(dataloaders, model, criterion, optimizer, device):
    losses = {}
    dice_coefficients = {}
    
    for phase in ["train", "val"]:
        running_loss = 0.0
        running_dice_coeff = 0.0
        
        if phase == "train":
            model.train()
        else:
            model.eval()
        
        for index, batch in enumerate(dataloaders[phase]):
            images = batch[0].to(device)
            targets = batch[1].to(device)
            
            with torch.set_grad_enabled(phase == "train"):
                predictions = model(images)
                loss, dice_coefficient = criterion(predictions, targets)
                
            if phase == "train":
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            running_loss += loss.item()
            running_dice_coeff += dice_coefficient.item()

            if index == 10: # 10 index * mini_batch 데이터수 만큼 데이터를 한정
                break

        losses[phase] = running_loss / index
        dice_coefficients[phase] = running_dice_coeff / index
        
    return losses, dice_coefficients

# 5. Weight Initialization 과 Transfer learning 모델 비교하기

## 5-1 He initialization

In [57]:
def He_initialization(module):
    if isinstance(module, nn.Conv2d):
        nn.init.kaiming_normal_(module.weight)
    elif isinstance(module, nn.BatchNorm2d):
        module.weight.data.fill_(1.0)

In [58]:
is_mps = True

NUM_CLASSES = 4
IMAGE_SIZE = 224
BATCH_SIZE = 12
DEVICE = torch.device('mps' if torch.backends.mps.is_available() and is_mps else 'cpu')

dataloaders = build_dataloader(data_dir=data_dir, batch_size=BATCH_SIZE)
model = UNet(num_classes=NUM_CLASSES, pretrained=False)
model.apply(He_initialization)
model = model.to(DEVICE)
criterion = UNet_metric(num_classes=NUM_CLASSES)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [59]:
num_epochs = 30

train_loss_def, train_dice_coefficient_def = [], []
val_loss_def, val_dice_coefficient_def = [], []

for epoch in range(num_epochs):
    losses, dice_coefficients = train_one_epoch(dataloaders, model, criterion, optimizer, DEVICE)
    train_loss_def.append(losses['train'])
    val_loss_def.append(losses['val'])
    train_dice_coefficient_def.append(dice_coefficients['train'])
    val_dice_coefficient_def.append(dice_coefficients['val'])
    
    print(f"{epoch}/{num_epochs} - Train loss: {losses['train']:.4f}, Val loss: {losses['val']:.4f}," + \
          f" Train dice: {dice_coefficients['train']:.4f}, Val dice: {dice_coefficients['val']:.4f}")

0/30 - Train loss: 2.3117, Val loss: 2.0145, Train dice: 0.0148, Val dice: 0.0056
1/30 - Train loss: 2.0234, Val loss: 1.8231, Train dice: 0.0114, Val dice: 0.0046
2/30 - Train loss: 1.8082, Val loss: 1.7219, Train dice: 0.0058, Val dice: 0.0030
3/30 - Train loss: 1.6929, Val loss: 1.6525, Train dice: 0.0025, Val dice: 0.0018
4/30 - Train loss: 1.6342, Val loss: 1.6133, Train dice: 0.0012, Val dice: 0.0011
5/30 - Train loss: 1.6022, Val loss: 1.5905, Train dice: 0.0007, Val dice: 0.0007
6/30 - Train loss: 1.5824, Val loss: 1.5759, Train dice: 0.0004, Val dice: 0.0006
7/30 - Train loss: 1.5687, Val loss: 1.5657, Train dice: 0.0003, Val dice: 0.0005
8/30 - Train loss: 1.5584, Val loss: 1.5579, Train dice: 0.0002, Val dice: 0.0004
9/30 - Train loss: 1.5502, Val loss: 1.5515, Train dice: 0.0002, Val dice: 0.0004
10/30 - Train loss: 1.5435, Val loss: 1.5463, Train dice: 0.0002, Val dice: 0.0004
11/30 - Train loss: 1.5378, Val loss: 1.5418, Train dice: 0.0002, Val dice: 0.0004
12/30 - Train 

## 5-2 Weight transfer pre-trained on ImageNet

In [61]:
dataloaders = build_dataloader(data_dir=data_dir, batch_size=BATCH_SIZE)
model = UNet(num_classes=NUM_CLASSES, pretrained=True)
model = model.to(DEVICE)
criterion = UNet_metric(num_classes=NUM_CLASSES)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /Users/jeong-wonseok/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [62]:
num_epochs = 30

train_loss_prt, train_dice_coefficient_prt = [], []
val_loss_prt, val_dice_coefficient_prt = [], []

for epoch in range(num_epochs):
    losses, dice_coefficients = train_one_epoch(dataloaders, model, criterion, optimizer, DEVICE)
    train_loss_prt.append(losses["train"])
    val_loss_prt.append(losses["val"])
    train_dice_coefficient_prt.append(dice_coefficients["train"])
    val_dice_coefficient_prt.append(dice_coefficients["val"])
    
    print(f"{epoch}/{num_epochs} - Train loss: {losses['train']:.4f}, Val loss: {losses['val']:.4f}," + \
          f" Train dice: {dice_coefficients['train']:.4f}, Val dice: {dice_coefficients['val']:.4f}")

0/30 - Train loss: 2.4013, Val loss: 2.4771, Train dice: 0.0426, Val dice: 0.0001
1/30 - Train loss: 2.0697, Val loss: 2.2692, Train dice: 0.0218, Val dice: 0.0000
2/30 - Train loss: 1.8071, Val loss: 1.9968, Train dice: 0.0015, Val dice: 0.0000
3/30 - Train loss: 1.6656, Val loss: 1.7577, Train dice: 0.0000, Val dice: 0.0000
4/30 - Train loss: 1.6008, Val loss: 1.6240, Train dice: 0.0000, Val dice: 0.0000
5/30 - Train loss: 1.5688, Val loss: 1.5640, Train dice: 0.0000, Val dice: 0.0000
6/30 - Train loss: 1.5499, Val loss: 1.5379, Train dice: 0.0000, Val dice: 0.0000
7/30 - Train loss: 1.5369, Val loss: 1.5252, Train dice: 0.0000, Val dice: 0.0000
8/30 - Train loss: 1.5272, Val loss: 1.5177, Train dice: 0.0000, Val dice: 0.0000
9/30 - Train loss: 1.5178, Val loss: 1.5127, Train dice: 0.0000, Val dice: 0.0000
10/30 - Train loss: 1.5092, Val loss: 1.5083, Train dice: 0.0000, Val dice: 0.0000
11/30 - Train loss: 1.5011, Val loss: 1.5039, Train dice: 0.0000, Val dice: 0.0000
12/30 - Train 

## 5-3 Weight transfer with freezing encoder layer

In [63]:
dataloaders = build_dataloader(data_dir=data_dir, batch_size=BATCH_SIZE)
model = UNet(num_classes=NUM_CLASSES, pretrained=True)
model = model.to(DEVICE)
model.encoder.requires_grad_ = False
criterion = UNet_metric(num_classes=NUM_CLASSES)
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9)

In [ ]:
num_epochs = 30

train_loss_frz, train_dice_coefficient_frz = [], []
val_loss_frz, val_dice_coefficient_frz = [], []

for epoch in range(num_epochs):
    losses, dice_coefficients = train_one_epoch(dataloaders, model, criterion, optimizer, DEVICE)
    train_loss_frz.append(losses["train"])
    val_loss_frz.append(losses["val"])
    train_dice_coefficient_frz.append(dice_coefficients["train"])
    val_dice_coefficient_frz.append(dice_coefficients["val"])
    
    print(f"{epoch}/{num_epochs} - Train loss: {losses['train']:.4f}, Val loss: {losses['val']:.4f}," + \
          f" Train dice: {dice_coefficients['train']:.4f}, Val dice: {dice_coefficients['val']:.4f}")

0/30 - Train loss: 2.2997, Val loss: 2.4209, Train dice: 0.0345, Val dice: 0.0000
1/30 - Train loss: 2.0216, Val loss: 2.1491, Train dice: 0.0098, Val dice: 0.0000
2/30 - Train loss: 1.7879, Val loss: 1.8848, Train dice: 0.0002, Val dice: 0.0000
3/30 - Train loss: 1.6630, Val loss: 1.6991, Train dice: 0.0000, Val dice: 0.0000
4/30 - Train loss: 1.6039, Val loss: 1.6033, Train dice: 0.0000, Val dice: 0.0000
5/30 - Train loss: 1.5733, Val loss: 1.5600, Train dice: 0.0000, Val dice: 0.0000
6/30 - Train loss: 1.5546, Val loss: 1.5394, Train dice: 0.0000, Val dice: 0.0000
7/30 - Train loss: 1.5415, Val loss: 1.5285, Train dice: 0.0000, Val dice: 0.0000
8/30 - Train loss: 1.5318, Val loss: 1.5203, Train dice: 0.0000, Val dice: 0.0000
9/30 - Train loss: 1.5239, Val loss: 1.5154, Train dice: 0.0000, Val dice: 0.0000
10/30 - Train loss: 1.5149, Val loss: 1.5103, Train dice: 0.0000, Val dice: 0.0000
11/30 - Train loss: 1.5069, Val loss: 1.5051, Train dice: 0.0000, Val dice: 0.0000
12/30 - Train 